# 8 속성 기반 검사

* 속성 기반 검사?? 그냥 스칼라에서 어떤 식으로 테스트 케이스를 만들고 활용하는지에 대한 설명
* 프로그래머가 테스트 방식을 명시하면, 프레임워크에서는 자동으로 테스트 케이스를 생성하고, 해당 케이스를 동작시켜서 프로그램이 잘 동작하는지를 테스트 해 준다.

## 8.1 ScalaCheck

* 유명한 스칼라 테스트 라이브러리

In [ ]:
val intList = Gen.listOf(Gen.choose(0,100))
val prop = forAll(intList)(l => l.reserse.reverse == l) && forAll(intList)(l => l.headOption == l.reverse.lastOption)
val failingProp = forAll(intList)(l => l.reverse == l)

In [ ]:
prop.check
failingProp.check

* test case minimization - 테스트가 실패하면 실패한 경우에 대해 가장 작은 경우에 도달할 때까지 테스트를 지속한다.
* exhausive test case generation - Gen이 생성할 수 있는 모든 경우에 대해 전수 검사를 실시할 수가 있다.(전수량이 작을 경우). 이를 통해 모든 경우에 대한 테스트가 성공이라는 것을 증명 할 수가 있다.

## 8.2 실제로 설계를 해보자

* 일단 우리가 구현해야 할 것은, intList와 prop
* intList의 경우 Gen[List[Int]] 형태의 값을 리턴한다.
* 형에 구현받지 않게 하기 위해 Gen[List[A]] 형태로 바꿔보자.
* def listOf[A](a: Gen[A]): Gen[List[A]]
* 크기를 입력받는 함수 또한 추가하자.
* def listOfN[A](n: Int, a: Gen[A]): Gen[List[A]]

* forAll의 경우도 아래와 같음을 확인하자
* def forAll[A](a: Gen[A])(f: A => Boolean):Prop
* Prop??

### Prop에 대해 자세히 알아보자

* forAll의 결과 값으로 Prop가 리턴됨을 확인할 수가 있다.
* Prop가 무엇인지는 잘 모르겠지만 check 함수와, && 연산자가 필요하단 것은 확실하다.

In [ ]:
trait Prop {
    def check: Unit
    def &&(p: Prop): Prop = { .... }
}

* Prop에서 &&연산자를 사용하기 위해서는 check의 결과 값을 통해 무언가를 확인할 수가 있어야 한다. (쉽게 말해서 true, false)
* 하지만 Boolean만으로는 부족한 추가적인 정보도 결과 값으로 제공하고 싶다. 가령 성공한 테스트 개수

In [ ]:
object Prop{
    type SuccessCount = Int
}
trait Prop { def check: Either[???, SuccessCount]}

* 실패의 경우에는 Either Left에 실패 메시지와 실패전까지의 성공 개수를 써주자.

In [ ]:
object Prop{
    type FailedCase = String
    type SuccessCount = Int
}

trait Prop {
    def check: Either[(FailedCase, SuccessCount), SuccessCount]
}

*  Prop를 좀더 다듬어보자. 몇개의 테스트가 성공해야 최종 성공했다는 판단을 할 것인지...

In [ ]:
type TestCases = Int
type Result = Either[(FailedCase, SuccessCount), SuccessCount]
case class Prop(run: TestCases => Result)

* Either도 한번 살펴보자.. Either right가 과연 필요한가?? option으로 바꿔도 되지 않을까?

In [ ]:
type Result = Option[(FailedCase, SuccessCount)]
case class Prop(run: TestCases => Result)

* 의미상으로 조금 이상하지만 Result가 None일 경우에는 Test 성공, Some이 나오면 Test 실패로 간주하면 된다. 문맥상 이상할 수 있으니 아래와 같이 바꿔보자

In [ ]:
sealed trait Result {
    def isFalsified: Boolean
}
case object Passed extends Result{
    def isFalsified = false
}
case class Falsified(failure: FailCase, successes: SuccessCount) extends Result{
    def isFalsified = true
}

* 전체 검수 검사 개수와 함께 테스트 범위도 파라미터로 필요하다.

In [ ]:
case class Prop(run: (TestCases,RNG) => Result)

In [ ]:
def forAll[A](as: Gen[A])(f: A => Boolean): Prop = Prop {
    (n,rng) => randomStream(as)(rng).zip(Stream.from(0)).take(n).map{
        case(a, i) => try{
            if (f(a)) Passed else Falsified(a.toString, i)
        }catch{case e: Exception => Falsified(buildMsg(a,e), i)}
    }.find(_.isFalsified).getOrElse(Passed)
}

def randomStream[A](g: Gen[A])(rng:RNG):Stream[A] = {
    Stream.unfold(rng)(rng => Some(g.sample.run(rng)))
}

def buildMsg[A](s: A, e: Exception): String = {
    s"test case: $s\n" +
    s"generated an exception: ${e.getMessage}\n" +
    s"stack trace:\n ${e.getStackTrace.mkString("\n")}"
}


## 8.3 Test case 최소화

* Test case를 최소화 하는 방법에는 두 가지가 있다.
* shrinking - 실패한 테스트가 나왔다면 그 테스트 크기를 점차 줄여나감으로써 범위를 줄이는 방법. ScalaCheck의 경우 이 방식을 사용한다.
* sized generation - 처음부터 다양한 크기의 테스트 케이스를 생성하여 테스트 하는 방식.

In [ ]:
case class SGen[+A](forSize: Int => Gen[A])
def forAll[A](g:SGen[A])(f:A=>Boolean):Prop

In [ ]:
* 위의 방식으로는 SGen을 사용할 수가 없다. Prop 에 Sgen에 대한 최대 크기를 정의를 해주어야 한다.

In [ ]:
type MaxSize = Int
case class Prop(run: (MaxSize, TestCases, RNG) => Result)

def forAll[A](g: Sgen[A])(f: A => Boolean): Prop = forAll(g(_))(f)

def forAll[A](g: Int => Gen[A])(f: A => Boolean): Prop = Prop {
    (max, n, rng) =>
    val casesPerSize = (n + (max -1)) / max
    val props: Stream[Prop] = Stream.from(0).take((n min max) + 1).map(i => forAll(g(i))(f))
    val prop: Prop = props.map(p => Prop{ (max, _, rng) => p.run(max, casesPerSize, rng)}).toList.reduce(_ && _)
    prop.run(max,n, rng)
}